In [ ]:
!pip install rembg
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import subprocess
from PIL import Image as PILImage, ImageEnhance
import cv2
import rembg

from google.colab import drive
drive.mount('/content/drive')

# Function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    """
    Params:
            js_reply: JavaScript object containing an image from the webcam
    Returns:
            img: OpenCV BGR image
    """
    # Decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # Convert bytes to a numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # Decode the numpy array into an OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)

    return img

# Function to convert an OpenCV Rectangle bounding box image into a base64 byte string to be overlayed on a video stream
def bbox_to_bytes(bbox_array):
    """
    Params:
            bbox_array: Numpy array (pixels) containing a rectangle to overlay on the video stream.
    Returns:
        bytes: Base64 image byte string
    """
    # Convert the array into a PIL image
    bbox_PIL = PILImage.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # Format the bounding box into PNG for return
    bbox_PIL.save(iobuf, format='png')
    # Format the return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

    return bbox_bytes

# Function to capture a photo and remove the background using rembg
def capture_and_remove_background(background_path):
    try:
        # Capture a photo
        def take_photo(filename='photo.jpg', quality=0.8):
            js = Javascript('''
              async function takePhoto(quality) {
                const div = document.createElement('div');
                const capture = document.createElement('button');
                capture.textContent = 'Capture';
                div.appendChild(capture);

                const video = document.createElement('video');
                video.style.display = 'block';
                const stream = await navigator.mediaDevices.getUserMedia({video: true});

                document.body.appendChild(div);
                div.appendChild(video);
                video.srcObject = stream;
                await video.play();

                // Resize the output to fit the video element.
                google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

                // Wait for Capture to be clicked.
                await new Promise((resolve) => capture.onclick = resolve);

                const canvas = document.createElement('canvas');
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                canvas.getContext('2d').drawImage(video, 0, 0);
                stream.getVideoTracks()[0].stop();
                div.remove();
                return canvas.toDataURL('image/jpeg', quality);
              }
              ''')
            display(js)

            # Get photo data
            data = eval_js('takePhoto({})'.format(quality))
            # Get the OpenCV format image
            img = js_to_image(data)

            cv2.imwrite(filename, img)

            return filename

        # Capture a photo and save only the object as 'photo.jpg'
        filename = take_photo('photo.jpg')
        print('Saved to {}'.format(filename))

        # Remove the background using rembg and overwrite 'photo.jpg' with the object
        remove_background('photo.jpg', 'photo.jpg')

        # Overlay the object on the specified background
        overlay_photo('photo.jpg', background_path)

        # Display the composed image with the new background
        display(Image('photo.jpg'))

    except Exception as err:
        # Handle any errors
        print(str(err))

# Function to remove the background using rembg
def remove_background(input_path, output_path):
    try:
        # Use rembg to remove the background
        subprocess.run(["rembg", input_path, output_path])
        print('Background removed and object saved to {}'.format(output_path))
    except Exception as err:
        print('Error:', str(err))

# Function to overlay the object on a different background
def overlay_photo(photo_path, background_path):
    try:
        # Open the captured image with rembg
        with PILImage.open(photo_path) as img:
            img = img.convert('RGBA')
            with rembg.remove(img) as output:
                output.save(photo_path, 'PNG')

        # Open the background image
        captured_image = PILImage.open(photo_path)
        background_image = PILImage.open(background_path)

        # Adjust contrast
        contrast = ImageEnhance.Contrast(captured_image)
        captured_image = contrast.enhance(1.0)  # You can adjust the factor (e.g., 2.0) as needed

        # Adjust sharpness
        sharpness = ImageEnhance.Sharpness(captured_image)
        captured_image = sharpness.enhance(2.0)  # You can adjust the factor (e.g., 2.0) as needed

        brightness = ImageEnhance.Brightness(background_image)
        background_image = brightness.enhance(0.7)

        captured_image = captured_image.resize(background_image.size)
        composed_image = PILImage.alpha_composite(background_image.convert('RGBA'), captured_image.convert('RGBA'))

        composed_image_path = f'composed_photo.png'
        composed_image.save(composed_image_path, 'PNG')

        print('Composed photo saved as {}'.format(composed_image_path))

    except Exception as err:
        print('Error:', str(err))

# Specify the path to your desired background image
background_image_path = '/content/drive/MyDrive/flask app/7.jpg'

# Capture a photo, remove the background, and overlay on the specified background
#capture_and_remove_background(background_image_path)

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time

#%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.keras import models


from tensorflow.python.keras import models

import os

from google.colab import drive
drive. mount('/content/drive')

def load_img(path_to_img):
  max_dim = 512
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)

  #img = kp_image.img_to_array(img)

  # We need to broadcast the image array such that it has a batch dimension
  img = np.expand_dims(img, axis=0)
  return img

def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)

def load_and_process_img(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

def deprocess_img(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")

  # perform the inverse of the preprocessing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

# Content layer where will pull our feature maps
content_layers = ['block5_conv2']

# Style layer we are interested in
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1'
               ]

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def get_model():
  """ Creates our model with access to intermediate layers.

  This function will load the VGG19 model and access the intermediate layers.
  These layers will then be used to create a new model that will take input image
  and return the outputs from these intermediate layers from the VGG model.

  Returns:
    returns a keras model that takes image inputs and outputs the style and
      content intermediate layers.
  """
  # Load our model. We load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  # Get output layers corresponding to style and content layers
  style_outputs = [vgg.get_layer(name).output for name in style_layers]
  content_outputs = [vgg.get_layer(name).output for name in content_layers]
  model_outputs = style_outputs + content_outputs
  # Build model
  return models.Model(vgg.input, model_outputs)

def get_content_loss(base_content, target):
  return tf.reduce_mean(tf.square(base_content - target))

def gram_matrix(input_tensor):
  # We make the image channels first
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]
  gram = tf.matmul(a, a, transpose_a=True)
  return gram / tf.cast(n, tf.float32)

def get_style_loss(base_style, gram_target):
  """Expects two images of dimension h, w, c"""
  # height, width, num filters of each layer
  # We scale the loss at a given layer by the size of the feature map and the number of filters
  height, width, channels = base_style.get_shape().as_list()
  gram_style = gram_matrix(base_style)

  return tf.reduce_mean(tf.square(gram_style - gram_target))# / (4. * (channels ** 2) * (width * height) ** 2)

def compute_loss(outputs, loss_weights, init_image, gram_style_matrices, content_features):
    style_weight, content_weight = loss_weights

    style_outputs = outputs[:num_style_layers]
    content_outputs = outputs[num_style_layers:]

    style_loss = tf.add_n([get_style_loss(style_output[0], target_gram) for style_output, target_gram in zip(style_outputs, gram_style_matrices)])

    content_loss = tf.add_n([get_content_loss(content_output[0], target_content) for content_output, target_content in zip(content_outputs, content_features)])

    style_loss *= style_weight
    content_loss *= content_weight

    total_loss = style_loss + content_loss

    return total_loss, style_loss, content_loss


def get_feature_representations(model, content_path, style_path):
  """Helper function to compute our content and style feature representations.

  This function will simply load and preprocess both the content and style
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers.

  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image

  Returns:
    returns the style features and the content features.
  """
  # Load our images in
  content_image = load_and_process_img(content_path)
  style_image = load_and_process_img(style_path)

  # batch compute content and style features
  style_outputs = model(style_image)
  content_outputs = model(content_image)


  # Get the style and content feature representations from our model
  style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
  content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]
  return style_features, content_features

def compute_loss_with_precomputed_style(style_features, model, loss_weights, init_image, content_features):
    style_weight, content_weight = loss_weights

    # Feed our init image through our model. This will give us the content representations.
    model_outputs = model(init_image)
    content_output_features = model_outputs[num_style_layers:]

    content_score = 0

    # Compute content loss from all layers
    weight_per_content_layer = 1.0 / float(num_content_layers)
    for target_content, comb_content in zip(content_features, content_output_features):
        content_score += weight_per_content_layer * get_content_loss(comb_content[0], target_content)

    content_score *= content_weight

    # Get total loss
    loss = content_score

    return loss, content_score


def compute_grads(cfg):
    with tf.GradientTape() as tape:
        outputs = cfg['model'](cfg['init_image'])
        all_loss = compute_loss(outputs, cfg['loss_weights'], cfg['init_image'], cfg['gram_style_features'], cfg['content_features'])
    total_loss = all_loss[0]
    return tape.gradient(total_loss, cfg['init_image']), all_loss


import IPython.display

def run_style_transfer(content_path,
                       style_path,
                       num_iterations=500,
                       content_weight=1e3,
                       style_weight=1e-2):
  # We don't need to (or want to) train any layers of our model, so we set their
  # trainable to false.
  model = get_model()
  for layer in model.layers:
    layer.trainable = False

  # Get the style and content feature representations (from our specified intermediate layers)
  style_features, content_features = get_feature_representations(model, content_path, style_path)
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]

  # Set initial image
  init_image = load_and_process_img(content_path)
  init_image = tf.Variable(init_image, dtype=tf.float32)
  # Create our optimizer
  opt = tf.optimizers.Adam(learning_rate=2, epsilon=1e-1)

  # For displaying intermediate images
  iter_count = 10

  # Store our best result
  best_loss, best_img = float('inf'), None

  # Create a nice config
  loss_weights = (style_weight, content_weight)
  cfg = {
    'model': model,  # Modeli cfg sözlüğüne ekleyin
    'loss_weights': loss_weights,
    'init_image': init_image,
    'gram_style_features': gram_style_features,
    'content_features': content_features
  }
  # For displaying
  num_rows = 2
  num_cols = 5
  display_interval = num_iterations/(num_rows*num_cols)
  start_time = time.time()
  global_start = time.time()

  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means

  imgs = []
  for i in range(num_iterations):
    grads, all_loss = compute_grads(cfg)
    loss, style_score, content_score = all_loss
    opt.apply_gradients([(grads, init_image)])
    clipped = tf.clip_by_value(init_image, min_vals, max_vals)
    init_image.assign(clipped)
    end_time = time.time()
    print(i)
    if loss < best_loss:
      # Update best loss and best image from total loss.
      best_loss = loss
      best_img = deprocess_img(init_image.numpy())

    if i % display_interval== 0:
      start_time = time.time()

      # Use the .numpy() method to get the concrete numpy array
      plot_img = init_image.numpy()
      plot_img = deprocess_img(plot_img)
      imgs.append(plot_img)
      IPython.display.clear_output(wait=True)
      IPython.display.display_png(Image.fromarray(plot_img))

      print('Iteration: {}'.format(i))
      print('Total loss: {:.4e}, '
            'style loss: {:.4e}, '
            'content loss: {:.4e}, '
            'time: {:.4f}s'.format(loss, style_score, content_score, time.time() - start_time))
  print('Total time: {:.4f}s'.format(time.time() - global_start))
  IPython.display.clear_output(wait=True)
  plt.figure(figsize=(14,4))
  for i,img in enumerate(imgs):
      plt.subplot(num_rows,num_cols,i+1)
      plt.imshow(img)
      plt.xticks([])
      plt.yticks([])

  return best_img, best_loss

from PIL import Image



def png_to_jpg(input_path, output_path, quality=95):

    try:

        image = Image.open(input_path)

        image = image.convert('RGB')  # PNG'nin şeffaf piksellerini beyaza dönüştürmek için

        image.save(output_path, 'JPEG', quality=quality)

        return True

    except Exception as e:

        print(f"Hata oluştu: {e}")

        return False



# Kullanımı:

#input_path = '/content/composed_photo.png'  # Dönüştürmek istediğiniz PNG dosyasının yolu

#output_path = 'cikis.jpg'  # Dönüşen JPG dosyasının kaydedileceği yol

#png_to_jpg(input_path, output_path)

#best, best_loss = run_style_transfer('/content/cikis.jpg',
#                                     '/content/drive/MyDrive/flask app/ulaş style.png', num_iterations=500)

#Image.fromarray(best)
def tensor_to_image(tensor):
  tensor = tensor
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

In [8]:
!pip install flask pyngrok
!pip install flask flask_ngrok
!ngrok authtoken 2Vfwbr8lLhZJjCQnGm8LB7KiXET_4JSFgnkiNKFt7RdTzszJs

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [9]:
!pip install qrcode

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, send_file, session
from flask_ngrok import run_with_ngrok
import os
import base64
import io
from PIL import Image
import qrcode

# Set the correct path to your project folder
os.chdir("/content/drive/MyDrive/flask app")

# Create a Flask app
app = Flask(__name__)
app.secret_key = 'can20011a'  # Set your secret key
app.static_folder = 'static'

# Set up ngrok for tunneling to access the app from the internet (only for Google Colab)
run_with_ngrok(app)

# Initialize background_image_path with a default value
app.config['background_image_path'] = None
# Initialize style_image_path with a default value
app.config['style_image_path'] = None


# Açılış sayfası
@app.route("/", methods=['GET', 'POST'])
def index():
      if request.method == 'POST':
          if request.form.get('button_a'):
              # A butonuna tıklandığında yapılacak işlemler burada
              return redirect(url_for('generate_qr_code_homepage'))

          elif request.form.get('button_b'):
              # B butonuna tıklandığında yapılacak işlemler burada
              return redirect(url_for('background'))  # home route'unu yönlendir

      return render_template('index.html')  # index.html'i göster


# Modify your Flask route to generate and serve the QR code image
@app.route("/generate_qr_code_homepage", methods=['GET'])
def generate_qr_code_homepage():
    # Get the current base URL where your Flask app is running
    base_url = request.host_url

    # Define the link you want to generate a QR code for (the homepage)
    link = base_url

    # Generate the QR code
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(link)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white")

    # Save the QR code image to a BytesIO object
    qr_code_image_io = io.BytesIO()
    img.save(qr_code_image_io, "PNG")
    qr_code_image_io.seek(0)

    # Convert the BytesIO object to a base64-encoded string
    qr_code_image_base64 = base64.b64encode(qr_code_image_io.read()).decode('utf-8')

    return render_template('index.html', qr_code_image=qr_code_image_base64)


  # Define a route for the home page
@app.route("/background", methods=['GET', 'POST'])
def home():
      if request.method == 'POST':
          try:
              # Get the selected background image path from the form
              background_path = request.form.get('background_path')

              # Store the selected background image path in the session
              session['background_image_path'] = background_path

              # Redirect to the style_selection route
              return redirect(url_for('style_selection'))
          except Exception as e:
              return f"Error: {str(e)}"

      # List all image files in the /static/Background directory
      background_images = []
      background_directory = "/content/drive/MyDrive/flask app/static/Background"
      for filename in os.listdir(background_directory):
          if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
              image_path = os.path.join(background_directory, filename)
              background_images.append(image_path)

      # Render the home.html template with background_images
      return render_template('home.html', background_images=background_images)

  # Define a route for style image selection
@app.route("/style_selection", methods=['GET', 'POST'])
def style_selection():
      if request.method == 'POST':
          try:
              # Get the selected style image path from the form
              style_path = request.form.get('style_path')

              # Store the selected style image path in the session
              session['style_image_path'] = style_path

              # Redirect to the capture route
              return redirect(url_for('capture'))
          except Exception as e:
              return f"Error: {str(e)}"

      # List all image files in the /static/Style Image directory for style images
      style_images = []
      style_directory = "/content/drive/MyDrive/flask app/static/Style Image"
      for filename in os.listdir(style_directory):
          if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
              image_path = os.path.join(style_directory, filename)
              style_images.append(image_path)

      # Render the style_selection.html template with style_images
      return render_template('style_selection.html', style_images=style_images)

@app.route("/capture", methods=['GET', 'POST'])
def capture():
      background_image_path = session.get('background_image_path', None)
      style_image_path = session.get('style_image_path', None)

      if request.method == 'POST' and 'image_data' in request.form:
          try:
              # Get the captured image data from the POST request
              image_data = request.form.get('image_data')

              # Check if image data is not empty
              if image_data:
                  # Decode the base64 image data
                  image_bytes = base64.b64decode(image_data.split(',')[1])

                  # Create a PIL image from the decoded bytes
                  image = Image.open(io.BytesIO(image_bytes))

                  # Save the captured image to a JPG file
                  captured_image_path = '/content/drive/MyDrive/flask app/captured_photos/captured_photo.jpg'
                  image.save(captured_image_path, 'JPEG')  # Save as JPEG format

                  # Check if both background_image_path and style_image_path are not None
                  if background_image_path and style_image_path:
                      # Remove the background and overlay it on the specified background
                      composed_photo_path = overlay_photo(captured_image_path, background_image_path)
                      png_to_jpg("/content/drive/MyDrive/flask app/composed_photo.png", "/content/drive/MyDrive/flask app/composed_photo.jpg", quality=95)
                      best, best_loss = run_style_transfer('/content/drive/MyDrive/flask app/composed_photo.jpg',
                                          style_image_path, num_iterations=500)

                    # Convert and save the stylized image
                      stylized_image_path = '/content/drive/MyDrive/flask app/static/stylized-image_new2.jpg'
                      Image.fromarray(best).save(stylized_image_path)

                      # Redirect to the result route after processing the image
                      return redirect(url_for('result'))

          except Exception as e:
              return f"Error: {str(e)}"

      # Render the capture.html template when the page is initially loaded or if there's no "Process" request
      return render_template('capture.html', background_image_path=background_image_path, style_image_path=style_image_path)


# Define a route to display the stylized image and provide a download link
@app.route("/result")
def result():
      stylized_image_path = '/content/drive/MyDrive/flask app/static/stylized-image_new2.jpg'

      # Render the result.html template with the stylized image path
      return render_template('result.html', stylized_image_path=stylized_image_path)

@app.route("/download_result")
def download_result():
      stylized_image_path = '/content/drive/MyDrive/flask app/static/stylized-image_new2.jpg'
      return send_file(stylized_image_path, as_attachment=True)

@app.route("/download_qr_code")
def download_qr_code():
      qr_code_file_path = '/content/drive/MyDrive/flask app/qr_codes/qr_code_homepage.png'
      return send_file(qr_code_file_path, as_attachment=True)

# Run the Flask app
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://041b-34-74-100-245.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Sep/2023 03:41:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Sep/2023 03:41:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [25/Sep/2023 03:41:14] "GET /generate_qr_code_homepage?button_a= HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Sep/2023 03:41:17] "GET /background?button_b= HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Sep/2023 03:41:19] "POST /background HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [25/Sep/2023 03:41:19] "GET /style_selection HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Sep/2023 03:41:22] "POST /style_selection HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [25/Sep/2023 03:41:22] "GET /capture HTTP/1.1" 200 -
